## Gerando arquivo para NER-NestedClinBr
Deixar sempre o arquivo de test para fazer o teste completo (com o pipeline todo)
1o. -> passar pelo modelo de NER normal (binario), pegar possiveis entidades
2o. -> combinacao de palavras, modelo de span

Descontinuas: treinar duplicando as frases, tirando os termos do meio.. na hora da predicao: se duas entidades estão mito proximas (ou no mesmo chunk), tirar o espaço e ver se são (ou tentar achar uma regra, ex sempre q tem tratamento e uma anotmia perto, usar uma janela.. ex duas palavras de distancia)...

Para tese: testar tbm sem as descontinuas.. pra pode comparar com outros metodos...


In [280]:
import os
from pathlib import Path
import re
import pickle
import random


In [281]:
def getTiposEntidade():
    return ['Problema','Teste','Tratamento','Anatomia']

In [282]:
def replaceWhiteSpaces(str):
    return re.sub('\s{2,}',' ',str)

In [283]:
def save_obj(name, obj):
    existeDir = os.path.exists('../obj')
    if not existeDir:
        os.makedirs('../obj')
    with open('../obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f)

def load_obj(name):
    existeDir = os.path.exists('../obj')
    if not existeDir:
        os.makedirs('../obj')
    try:
        with open('../obj/' + name + '.pkl', 'rb') as f:
            return pickle.load(f)
    except:
        return None

In [288]:
def getIndices(listaFrase, termos, indiceInicio):
    sep='#SEP#'
    listaFrase=sep.join(listaFrase)
    tam_original=len(termos)
    termos = termos.replace(' ',sep)
    indices = [i.start() for i in re.finditer(re.escape(termos), listaFrase)]
    tam=len(termos)
    lista_indices = []
    for indice in indices:
        #print('indice:', indice)
        inicioFrase=listaFrase[0:indice+tam]
        numSEP = len(inicioFrase.split(sep))
        inicio = numSEP - len(termos.split(sep))
        fim = numSEP-1
        if inicio>=indiceInicio:
            for i in range(inicio,fim+1):
                lista_indices.append(i)
            indiceFim = indice + tam_original
            break

    return lista_indices

getIndices(['oi','camarada','tudo','bem', 'com', 'vc', 'tudo', 'bem','e', 'vc'], 'tudo bem', 5)

[6, 7]

In [289]:
pastasCorpus = [r'corpus\test',r'corpus\train']

In [294]:
def getDicSentences(pastaCorpus):
    dic_sentences = {}
    num=0
    listaEntidades=[]
    tokens=[]
    frasesComDescontinuas=[]
    for filename in os.listdir(pastaCorpus):
        f = os.path.join(pastaCorpus, filename)
        if os.path.isfile(f):
            fileNameSemExtensao=os.path.splitext(filename)[0]
            #print('\n\n--fileName (sem extensao):--', fileNameSemExtensao)
            extension = os.path.splitext(filename)[1][1:]
            if extension=='ann':
                # tokens da frase
                fileTxt = open(os.path.join(pastaCorpus, fileNameSemExtensao)+'.txt', "r", encoding='utf-8')
                linha=fileTxt.readlines()
                fileTxt.close()
                fraseOriginal=linha[0]
                #print('fraseOriginal:', fraseOriginal)
                linha = linha[0]
                # para tokenizar nesses tokens
                linha = linha.strip().replace('/',' / ').replace(')',' ) ').replace('(',' ( ').replace(']',' ] ').replace('[',' [ ').replace(',',' , ').replace('.',' . ').replace(';',' ; ').replace('-',' - ').replace('+',' + ').replace("'"," ' ").replace(" = "," = ").replace("#"," # ").replace(" $ "," $ ").replace(" ! "," ! ").replace("?"," ? ").replace("%"," % ").replace(":"," : ").replace(">"," > ").replace("<"," < ")
                linha = replaceWhiteSpaces(linha)
                linha = linha.split()
                for numtoken, token in enumerate(linha):
                    tokens.append([token,numtoken])
                linhaTokens=linha.copy()    
                # agora, as entidades
                fileAnn = open(f, "r", encoding='utf-8')
                linha=fileAnn.readlines()
                fileAnn.close()

                dicEntidades={}
                for entidade_linha in linha:
                    if ';' not in entidade_linha: # tem descontinua?
                        entidade = entidade_linha.split('\t')
                        tipo_entidade = entidade[1]
                        inicio, fim = tipo_entidade.split()[1:3]
                        tipo_entidade = tipo_entidade.split()[0]
                        termos_entidade = entidade[2].replace('\n','')
                        termos_entidade = termos_entidade.strip().replace('/',' / ').replace(')',' ) ').replace('(',' ( ').replace(']',' ] ').replace('[',' [ ').replace(',',' , ').replace('.',' . ').replace(';',' ; ').replace('-',' - ').replace('+',' + ').replace("'"," ' ").replace(" = "," = ").replace("#"," # ").replace(" $ "," $ ").replace(" ! "," ! ").replace("?"," ? ").replace("%"," % ").replace(":"," : ").replace(">"," > ").replace("<"," < ")
                        dicEntidades[(int(inicio), int(fim))]=[tipo_entidade, replaceWhiteSpaces(termos_entidade)]
                    else:
                        frasesComDescontinuas.append(num)
                        #print('descontinua')
                        entidade = entidade_linha.split('\t')
                        # ex T10	Problema 244 252;279 306	dispneia aos mdoeardos-leves esforço
                        #Problema 244 252;279 306
                        entidade_temp=entidade[1].split(';')
                        entidade1=entidade_temp[0]
                        tipo_entidade = entidade1
                        inicio1, fim1 = tipo_entidade.split()[1:3]
                        tipo_entidade_string = tipo_entidade.split()[0]
                        # TODO - mandar só os termos referentes...
                        tamTermo1=int(fim1)-int(inicio1)
                        termos_entidade = entidade[2].replace('\n','')
                        termos_entidade=termos_entidade[:tamTermo1]
                        termos_entidade = termos_entidade.strip().replace('/',' / ').replace(')',' ) ').replace('(',' ( ').replace(']',' ] ').replace('[',' [ ').replace(',',' , ').replace('.',' . ').replace(';',' ; ').replace('-',' - ').replace('+',' + ').replace("'"," ' ").replace(" = "," = ").replace("#"," # ").replace(" $ "," $ ").replace(" ! "," ! ").replace("?"," ? ").replace("%"," % ").replace(":"," : ").replace(">"," > ").replace("<"," < ")
                        dicEntidades[(int(inicio1), int(fim1))]=[tipo_entidade_string, replaceWhiteSpaces(termos_entidade)]

                        entidade2=entidade_temp[1]
                        inicio2, fim2 = entidade2.split()[0:2]
                        termos_entidade = entidade[2].replace('\n','')
                        termos_entidade=termos_entidade[tamTermo1:len(termos_entidade)]
                        termos_entidade = termos_entidade.strip().replace('/',' / ').replace(')',' ) ').replace('(',' ( ').replace(']',' ] ').replace('[',' [ ').replace(',',' , ').replace('.',' . ').replace(';',' ; ').replace('-',' - ').replace('+',' + ').replace("'"," ' ").replace(" = "," = ").replace("#"," # ").replace(" $ "," $ ").replace(" ! "," ! ").replace("?"," ? ").replace("%"," % ").replace(":"," : ").replace(">"," > ").replace("<"," < ")
                        dicEntidades[(int(inicio2), int(fim2))]=[tipo_entidade_string, replaceWhiteSpaces(termos_entidade)]


                #print(dicEntidades)
                indicesDic = sorted(dicEntidades.keys(), key = lambda item: item[0])
                #list_students.sort(key = lambda x: x[1])   #index 1 means second element
                #print(indicesDic)

                indiceAnterior=0
                for i in indicesDic:
                    tipo_entidade,termos_entidade = dicEntidades[i]
                    #print('tipo_entidade:',tipo_entidade)
                    #print('linhaTokens:',linhaTokens)
                    #print('termos_entidade:',termos_entidade)
                    #print('indiceAnterior:', indiceAnterior)
                    novos_indices=getIndices(linhaTokens, termos_entidade, indiceAnterior)
                    #print('novos_indices:', novos_indices)
                    indiceAnterior=novos_indices[0]
                    #idx = sentence.index("x")
                    listaEntidades.append([termos_entidade, novos_indices, tipo_entidade])

            if len(tokens)>0:
                dic_sentences[num]=[tokens, listaEntidades]
                listaEntidades=[]
                tokens=[]
                num=num+1
            #if num>10:
            #    break
    return dic_sentences, frasesComDescontinuas
            


In [305]:
print('--Treinamento:--', pastasCorpus[1])
dic_sentencesTrain, frasesComDescontinuasTrain = getDicSentences(pastasCorpus[1])
print('len(sentences):', len(sentencesTrain))
print('len(Descontinuas):',len(frasesComDescontinuasTrain))
print('len(frasesComDescontinuas):',len(set(frasesComDescontinuasTrain)))

print('--Teste:--', pastasCorpus[0])
dic_sentencesTest, frasesComDescontinuasTest = getDicSentences(pastasCorpus[0])
print('len(sentences):', len(sentencesTest))
print('len(Descontinuas):',len(frasesComDescontinuasTest))
print('len(frasesComDescontinuas):',len(set(frasesComDescontinuasTest)))


--Treinamento:-- corpus\train
len(sentences): 100
len(Descontinuas): 91
len(frasesComDescontinuas): 51
--Teste:-- corpus\test
len(sentences): 26
len(Descontinuas): 44
len(frasesComDescontinuas): 17


In [306]:
frasesComDescontinuas[:10]

[1, 2, 3, 3, 3, 3, 5, 5, 6, 6]

In [307]:
# frases com entidades descontinuas
print(len(set(frasesComDescontinuas)))

51


In [308]:
print(dic_sentences[1])

[[['Data', 0], ['de', 1], ['Criação', 2], ['do', 3], ['Documento', 4], [':', 5], ['06', 6], ['/', 7], ['05', 8], ['/', 9], ['2016', 10], ['Plastia', 11], ['Mitral', 12], ['(', 13], ['Insuficiencia', 14], [')', 15], [',', 16], ['CRM', 17], ['Saf', 18], ['-', 19], ['2Mg', 20], ['e', 21], ['e', 22], ['Saf', 23], ['-', 24], ['3MG', 25], [')', 26], ['.', 27], ['(', 28], ['09', 29], ['/', 30], ['03', 31], ['/', 32], ['16', 33], [')', 34], ['.', 35], ['Uso', 36], [':', 37], ['AAS', 38], ['100', 39], ['-', 40], ['1xd', 41], [';', 42], ['Metoprolol', 43], ['25', 44], ['-', 45], ['1xd', 46], [';', 47], ['FSM', 48], ['-', 49], ['1xd', 50], [';', 51], ['Levotiroxina', 52], ['175', 53], ['-', 54], ['1xd', 55], [';', 56], ['Sinva', 57], ['40', 58], ['-', 59], ['1xd', 60], [';', 61], ['Fluoxetina', 62], ['20', 63], ['-', 64], ['1xd', 65], ['.', 66], ['Refere', 67], ['melhora', 68], ['da', 69], ['dispneia', 70], ['depois', 71], ['da', 72], ['cx', 73], ['porem', 74], ['mantem', 75], ['aos', 76], ['mdoe

In [311]:
save_obj('dic_sentencesTrain',dic_sentencesTrain)
save_obj('dic_sentencesTest',dic_sentencesTest)

In [339]:
# gerar arquivo treinamento - frases com mais de 100 caracteres sao quebradas em duas
f_train = open('nested_train.conll', 'w')

quebraPorFrase=True

num_entidade_total=0
num_entidade_train=0
num_entidade_test=0

tamMax=80
numVezesPassouTamMax=0

for i in range(len(dic_sentencesTrain)):
    tokens = dic_sentencesTrain[i][0]
    ents = dic_sentencesTrain[i][1]
    indiceEnts=[]
    numTokens=0
    
    tokensContinuacao=[]
    for token in tokens:
        numTokens+=1
        #print('token:', token)
        indiceToken = token[1]
        tag='O'
        for ent in ents:
            if indiceToken in ent[1]:
                tag = ent[2]
                break
        tokenGravar = token[0].replace(' ','')
        tokenGravar = tokenGravar.strip()
        if numTokens>tamMax:
            f_train.write('\n')
            numTokens=0
            numVezesPassouTamMax=numVezesPassouTamMax+1
        f_train.write(tokenGravar+' '+tag+'\n')
        num_entidade_train=num_entidade_train+1
        if quebraPorFrase:
            if tokenGravar=='.':
                f_train.write('\n')
    if not quebraPorFrase:
        f_train.write('\n')

        quebraFrase100
        
f_train.close()

f_test = open('nested_test.conll', 'w')
for i in range(len(dic_sentencesTest)):
    tokens = dic_sentencesTest[i][0]
    ents = dic_sentencesTest[i][1]
    indiceEnts=[]
    numTokens=0
    for token in tokens:
        numTokens+=1
        #print('token:', token)
        indiceToken = token[1]
        tag='O'
        for ent in ents:
            if indiceToken in ent[1]:
                tag = ent[2]
                break
        tokenGravar = token[0].replace(' ','')
        tokenGravar = tokenGravar.strip()
        if numTokens>tamMax:
            f_test.write('\n')
            numTokens=0
            numVezesPassouTamMax=numVezesPassouTamMax+1
        f_test.write(tokenGravar+' '+tag+'\n')
        num_entidade_test=num_entidade_test+1
        if quebraPorFrase:
            if tokenGravar=='.':
                f_test.write('\n')
    if not quebraPorFrase:
        f_test.write('\n')
f_test.close()

print('num_entidade_train', num_entidade_train)
print('num_entidade_test:', num_entidade_test)
num_entidade_total=num_entidade_train+num_entidade_test
print('num_entidade_total:', num_entidade_total)
print('numVezesPassouTamMax:', numVezesPassouTamMax)

save_obj('dic_sentencesTrain',dic_sentencesTrain)
save_obj('dic_sentencesTest',dic_sentencesTest)


num_entidade_train 17980
num_entidade_test: 5547
num_entidade_total: 23527
numVezesPassouTamMax: 231


In [ ]:
# gerar arquivo treinamento
f_train = open('nested_train.conll', 'w')

quebraPorFrase=True
numFrasesMaiores100=[]

num_entidade_total=0
num_entidade_train=0
num_entidade_test=0

for i in range(len(dic_sentencesTrain)):
    tokens = dic_sentencesTrain[i][0]
    ents = dic_sentencesTrain[i][1]
    indiceEnts=[]
    numTokens=0
    for token in tokens:
        numTokens+=1
        #print('token:', token)
        indiceToken = token[1]
        tag='O'
        for ent in ents:
            if indiceToken in ent[1]:
                tag = ent[2]
                break
        tokenGravar = token[0].replace(' ','')
        tokenGravar = tokenGravar.strip()
        if not numTokens>300:
            f_train.write(tokenGravar+' '+tag+'\n')
            num_entidade_train=num_entidade_train+1
            if quebraPorFrase:
                if tokenGravar=='.':
                    f_train.write('\n')
    if not quebraPorFrase and not numTokens>300:
        f_train.write('\n')
    if numTokens>100:
        numFrasesMaiores100.append(numTokens)
        
f_train.close()

f_test = open('nested_test.conll', 'w')
for i in range(len(dic_sentencesTest)):
    tokens = dic_sentencesTest[i][0]
    ents = dic_sentencesTest[i][1]
    indiceEnts=[]
    numTokens=0
    for token in tokens:
        numTokens+=1
        #print('token:', token)
        indiceToken = token[1]
        tag='O'
        for ent in ents:
            if indiceToken in ent[1]:
                tag = ent[2]
                break
        tokenGravar = token[0].replace(' ','')
        tokenGravar = tokenGravar.strip()
        if not numTokens>300:
            f_test.write(tokenGravar+' '+tag+'\n')
            num_entidade_test=num_entidade_test+1
            if quebraPorFrase:
                if tokenGravar=='.':
                    f_test.write('\n')
    if not quebraPorFrase and not numTokens>300:
        f_test.write('\n')
    if numTokens>100:
        numFrasesMaiores100.append(numTokens)
f_test.close()

print('num_entidade_train', num_entidade_train)
print('num_entidade_test:', num_entidade_test)
num_entidade_total=num_entidade_train+num_entidade_test
print('num_entidade_total:', num_entidade_total)


save_obj('dic_sentencesTrain',dic_sentencesTrain)
save_obj('dic_sentencesTest',dic_sentencesTest)


In [335]:
numFrasesMaiores100.sort(reverse=True)
numFrasesMaiores100

[412,
 336,
 335,
 334,
 333,
 332,
 328,
 322,
 315,
 312,
 311,
 310,
 301,
 299,
 296,
 290,
 290,
 290,
 290,
 289,
 286,
 274,
 272,
 272,
 268,
 268,
 265,
 264,
 260,
 258,
 257,
 256,
 255,
 254,
 253,
 252,
 252,
 251,
 250,
 248,
 237,
 235,
 230,
 227,
 220,
 218,
 213,
 212,
 208,
 206,
 203,
 202,
 201,
 198,
 196,
 193,
 192,
 192,
 191,
 191,
 188,
 181,
 177,
 176,
 175,
 170,
 169,
 165,
 164,
 163,
 163,
 163,
 163,
 163,
 161,
 160,
 159,
 159,
 158,
 157,
 157,
 152,
 147,
 138,
 137,
 137,
 131,
 131,
 131,
 131,
 128,
 117,
 116,
 111,
 107,
 106,
 104,
 103,
 103,
 103,
 102,
 102]

In [ ]:
# BKP
# gerar arquivo treinamento
f_train = open('nested_train.conll', 'w')

quebraPorFrase=True

num_entidade_total=0
num_entidade_train=0
num_entidade_test=0

for i in range(len(dic_sentencesTrain)):
    tokens = dic_sentencesTrain[i][0]
    ents = dic_sentencesTrain[i][1]
    indiceEnts=[]
    for token in tokens:
        #print('token:', token)
        indiceToken = token[1]
        tag='O'
        for ent in ents:
            if indiceToken in ent[1]:
                tag = ent[2]
                break
        tokenGravar = token[0].replace(' ','')
        tokenGravar = tokenGravar.strip()
        f_train.write(tokenGravar+' '+tag+'\n')
        num_entidade_train=num_entidade_train+1
        if quebraPorFrase:
            if tokenGravar=='.':
                f_train.write('\n')
    if not quebraPorFrase:
        f_train.write('\n')
        
f_train.close()

f_test = open('nested_test.conll', 'w')
for i in range(len(dic_sentencesTest)):
    tokens = dic_sentencesTest[i][0]
    ents = dic_sentencesTest[i][1]
    indiceEnts=[]
    for token in tokens:
        #print('token:', token)
        indiceToken = token[1]
        tag='O'
        for ent in ents:
            if indiceToken in ent[1]:
                tag = ent[2]
                break
        tokenGravar = token[0].replace(' ','')
        tokenGravar = tokenGravar.strip()
        f_test.write(tokenGravar+' '+tag+'\n')
        num_entidade_test=num_entidade_test+1
        if quebraPorFrase:
            if tokenGravar=='.':
                f_test.write('\n')
    if not quebraPorFrase:
        f_test.write('\n')
f_test.close()

print('num_entidade_train', num_entidade_train)
print('num_entidade_test:', num_entidade_test)
num_entidade_total=num_entidade_train+num_entidade_test
print('num_entidade_total:', num_entidade_total)

save_obj('dic_sentencesTrain',dic_sentencesTrain)
save_obj('dic_sentencesTest',dic_sentencesTest)


## parte 2- tentar tirar verbos, CC, etc do O para nao ficar com mto FP

In [700]:
# 1o. encontrar os postaggers das entidades...
import xml.dom.minidom

def getDicPosTagger(XMLfileName):
    dicPostagger = {}
    doc = xml.dom.minidom.parse(XMLfileName);
    sentences = doc.getElementsByTagName("sentence")
    for sentence in sentences:
        # ver se tem entidade do interesse
        cons = sentence.getElementsByTagName("cons")
        list_w = sentence.getElementsByTagName("w")
        for i, w in enumerate(list_w):
            texto = w.firstChild.data.lower()
            postagger = w.getAttribute("c")
            if (postagger == '*'):
                postagger = 'NN'
            dicPostagger[texto] = postagger
            for palavra in texto.split('-'):
                if not palavra in dicPostagger.keys():
                    dicPostagger[palavra] = postagger
            for palavra in texto.split('/'):
                if not palavra in dicPostagger.keys():
                    dicPostagger[palavra] = postagger
            if not texto.replace('(','').replace(')','') in dicPostagger.keys():
                dicPostagger[texto] = postagger
    #save_obj('dicPostagger', dicPostagger)
    return dicPostagger

pathCorpus=r'GENIAcorpus3.02.merged_corrigido.xml'
dicPosTagger = getDicPosTagger(pathCorpus)

In [706]:
def tipoPostaggerTokens(entidade_token, dicPostagger):
    postagger = ''
    for p in entidade_token:
        if p.lower() in dicPostagger.keys():
            postagger = postagger + '-' + dicPostagger.get(p.lower())
        elif p.lower().replace(',','') in dicPostagger.keys():
            postagger = postagger + '-' + dicPostagger.get(p.lower().replace(',',''))
        elif p.lower().replace('(','').replace(')','') in dicPostagger.keys():
            postagger = postagger + '-' + dicPostagger.get(p.lower().replace('(','').replace(')',''))
        else:
            for palavra in p.lower().split('/'):
                if palavra in dicPostagger.keys():
                    postagger = postagger + '-' + dicPostagger.get(palavra)
            for palavra in p.lower().split('-'):
                if palavra in dicPostagger.keys():
                    postagger = postagger + '-' + dicPostagger.get(palavra)
    return postagger

tipoPostaggerTokens(['IL','-','2','gene','CD28'], dicPosTagger)

'-NN-NN-CD-NN-NN'

In [732]:
lista_postaggers_entidades = []
for key, value in dicSentences.items():
    entidades = value[1]
    #print(entidades)
    for entidade in entidades:
        pos_tagger=tipoPostaggerTokens(entidade[0], dicPosTagger)
        if pos_tagger=='-JJ':
            print(entidade)
            print(value[0])
        if pos_tagger not in lista_postaggers_entidades:
            lista_postaggers_entidades.append(pos_tagger)
    


[['bHLH'], [10], 'PROTEIN']
[['It', 0], ['contains', 1], ['a', 2], ['basic', 3], ['helix', 4], ['-', 5], ['loop', 6], ['-', 7], ['helix', 8], ['(', 9], ['bHLH', 10], [')', 11], ['motif', 12], ['contiguous', 13], ['with', 14], ['a', 15], ['Per', 16], ['-', 17], ['Arnt', 18], ['-', 19], ['Sim', 20], ['(', 21], ['PAS', 22], [')', 23], ['homology', 24], ['region', 25], ['.', 26]]
[['GSSG'], [16], 'PROTEIN']
[['Because', 0], ['inhibition', 1], ['of', 2], ['DNA', 3], ['-', 4], ['binding', 5], ['activity', 6], ['is', 7], ['associated', 8], ['with', 9], ['increasing', 10], ['intracellular', 11], ['glutathione', 12], ['disulfide', 13], ['levels', 14], ['and', 15], ['GSSG', 16], ['can', 17], ['be', 18], ['shown', 19], ['to', 20], ['inhibit', 21], ['the', 22], ['DNA', 23], ['-', 24], ['binding', 25], ['activity', 26], ['directly', 27], ['in', 28], ['cell', 29], ['-', 30], ['free', 31], ['systems', 32], [',', 33], ['our', 34], ['studies', 35], ['suggest', 36], ['that', 37], ['GSSG', 38], ['is', 39

[['myeloblast'], [20], 'CELL_TYPE']
[['Egr', 0], ['-', 1], ['1', 2], ['antisense', 3], ['oligomers', 4], ['in', 5], ['the', 6], ['culture', 7], ['media', 8], ['blocked', 9], ['macrophage', 10], ['differentiation', 11], ['in', 12], ['both', 13], ['myeloid', 14], ['leukemia', 15], ['cell', 16], ['lines', 17], ['and', 18], ['normal', 19], ['myeloblasts', 20], ['.', 21]]
[['myeloblast'], [16], 'CELL_TYPE']
[['These', 0], ['observations', 1], ['indicate', 2], ['that', 3], ['expression', 4], ['of', 5], ['Egr', 6], ['-', 7], ['1', 8], ['is', 9], ['essential', 10], ['for', 11], ['and', 12], ['restricts', 13], ['differentiation', 14], ['of', 15], ['myeloblasts', 16], ['along', 17], ['the', 18], ['macrophage', 19], ['lineage', 20], ['.', 21]]
[['PRE'], [5], 'DNA']
[['The', 0], ['enhancer', 1], ['activity', 2], ['of', 3], ['the', 4], ['PRE', 5], ['was', 6], ['completely', 7], ['suppressed', 8], ['when', 9], ['the', 10], ['NRE', 11], ['was', 12], ['present', 13], ['.', 14]]
[['hepatic', 'microsome

In [719]:
lista_postaggers_entidades[:10]

['-NN-NN-CD-NN',
 '-NN-NN-NN-NN',
 '-NN',
 '-CD-NN-NN',
 '-NN-NN-NN',
 '-NN-NN-CD',
 '-JJ-NN-NNS',
 '-NN-NN-JJ-NN',
 '-NN-NN',
 '-NN-NN-NN-NN-NN']

In [725]:
print('-CD' in lista_postaggers_entidades)
print('-CC' in lista_postaggers_entidades) # -> só tem 2, um é em maiusculo, excluir
print('-JJ' in lista_postaggers_entidades)
print('-VBN' in lista_postaggers_entidades)
print('-IN' in lista_postaggers_entidades)
print('-TO' in lista_postaggers_entidades)
print('-VBZ' in lista_postaggers_entidades)
print('-NNS' in lista_postaggers_entidades)

True
True
True
False
False
False
False
True


In [832]:
palavrasDescontinua_underline=['AND_NOT', 'AND/OR', 'AS_WELL_AS', 'AND', 'OR', 'BUT_NOT', 'NEITHER_NOR', 'THAN']
num=0
numDescontinuas=0
erro_corpus=0
num_frases_sem_entidade=0
lista_erro_corpus=list()
combinacaoEntidadesAll = list()
combinacaoEntidades = list()
pulando_termos_postagger = list()
for key, value in dicSentencesComLabel.items():
    num=num+1
    combinacaoEntidades = list()
    #print('key:', key)
    #print(value)
    tokens=value[0].copy()
    so_tokens = [t[0] for t in tokens]
    entidades=value[1].copy()
    #print(so_tokens)
    #print(entidades)
    for entidade in entidades:
        erros_entidade = list()
        #print(entidade[1])
        texto_entidade=entidade[0]
        indices = entidade[1]
        tipo_entidade = entidade[2]
        if areConsecutive(indices): # ver se não é descontinua
            #print(entidade[1])
            #print(frase)
            frase = so_tokens.copy()
            inicio=indices[0]
            fim=indices[-1]
            frase.insert(inicio, '<e1>')
            frase.insert(fim+2, '</e1>')
            if ' '.join(texto_entidade).strip()=='-' or ' '.join(texto_entidade).strip()=='=' or ' '.join(texto_entidade).strip()=='+' or ' '.join(texto_entidade).strip()==':' or ' '.join(texto_entidade).strip()==',' or ' '.join(texto_entidade).strip()=="'" or ' '.join(texto_entidade).strip()=='"' or ' '.join(texto_entidade).strip()=='.' or ' '.join(texto_entidade).strip()==';' or ' '.join(texto_entidade).strip()=='/' or ' '.join(texto_entidade).strip()=='(' or ' '.join(texto_entidade).strip()==')' or ' '.join(texto_entidade).strip()=='[' or ' '.join(texto_entidade).strip()==']':
                pass
            #print('--texto_entidade--:', texto_entidade)
            #print('frase:', frase)
            #print('frase[inicio+1:fim+2]:', frase[inicio+1:fim+2])
            #print(tokens[indice])
            # o corpus tem alguns problemas, ex tem entidade descontinua sem o CC
            # entao aqui, se não bater, não adicionar no arquivo de treinamento
            # ex frase "The inhibition of c - fos and c - jun expression by IL - 4 in LPS - treated cells was shown to be due to a lower transcription rate of the c - fos and c - jun genes ."
            #print("texto_entidade:", texto_entidade)
            #print("frase[inicio:fim]:", frase[inicio+1:fim+2])
            texto_entidade_comparar=' '.join(texto_entidade).strip().replace('/','').replace(')','').replace('(','').replace(']','').replace('[','').replace(',','').replace('.','').replace(';','').replace('-','').replace('+','').replace("'",'')
            texto_entidade_comparar = replaceWhiteSpaces(texto_entidade_comparar)
            texto_frase_comparar = ' '.join(frase[inicio+1:fim+2]).strip().replace('/','').replace(')','').replace('(','').replace(']','').replace('[','').replace(',','').replace('.','').replace(';','').replace('-','').replace('+','').replace("'",'')
            texto_frase_comparar = replaceWhiteSpaces(texto_frase_comparar)
            #tem = [palavra.lower() in texto_entidade_comparar for palavra in palavrasDescontinua]
            #tem = [' '+palavra.lower()+' ' in ' '+texto_entidade_comparar+' ' for palavra in palavrasDescontinua]
            #texto_frase_comparar2 = texto_frase_comparar.replace('and not', 'and_not').replace('as well as', 'as_well_as').replace('neither nor', 'neither_nor').replace('but not', 'but_not')
            #print(frase)
            #print('texto_frase_comparar2.split():', texto_frase_comparar2.split())
            #print('texto_frase_comparar2.split():', texto_frase_comparar2.split()[0])
            #tem = [texto_frase_comparar2.split()[-1]==palavra.lower() for palavra in palavrasDescontinua_underline]
            #tem = [' '+palavra.lower()+' ' in ' '+texto_frase_comparar2+' ' for palavra in palavrasDescontinua_underline]
            #tem_boolean = True in tem
            #print([' '.join(frase).strip(), tipo_entidade])
            #if not tem_boolean:
            #if 1==2:
            #    combinacaoEntidades.append([' '.join(frase).strip(), tipo_entidade]) # apendando entidades reais
            texto_entidade_comparar = texto_entidade_comparar.lower()
            texto_frase_comparar = texto_frase_comparar.lower()
            if ((texto_entidade_comparar == texto_frase_comparar) or (texto_entidade_comparar+'s' == texto_frase_comparar) or (texto_entidade_comparar+'es' == texto_frase_comparar) or (texto_entidade_comparar+'ies' == texto_frase_comparar) or (texto_entidade_comparar[:-1]+'ies' == texto_frase_comparar) or (texto_entidade_comparar[:-1] == texto_frase_comparar.replace(' /','/'))):
                combinacaoEntidades.append([' '.join(frase).strip(), tipo_entidade]) # apendando entidades reais
            else:
                #print('tem:', tem)
                #print('texto_entidade_comparar:', texto_entidade_comparar)
                #print('texto_frase_comparar:', texto_frase_comparar)
                #print('texto_frase_comparar2.split()[-1]:', texto_frase_comparar2.split()[-1])
                #print("' '.join(texto_entidade).strip()):", ' '.join(texto_entidade).strip())
                #print("' '.join(frase[inicio:fim].strip():", ' '.join(frase[inicio+1:fim+2]).strip())
                erro_corpus=erro_corpus+1
                erros_entidade.append(indices)
                #print('-----indices erro:----', indices)
                lista_erro_corpus.append([' '.join(frase).strip(), tipo_entidade, ' '.join(so_tokens), entidade])
        else: # agora, qdo são descontinuas
            numDescontinuas=numDescontinuas+1
            #print(entidade[1])
            #frase = so_tokens.copy()
            #inicio=indices[0]
            #fim=indices[-1]
            #frase.insert(inicio, '<e1>')
            #frase.insert(fim+2, '</e1>')
            
    for entidade in entidades:
            indices = entidade[1]
            #print('indices:', indices)
            if indices in erros_entidade:
                continue
            inicio=indices[0]
            fim=indices[-1]
            # agora, fazer a combinacao entre eles.. todas a seguir serão do tipo 'O'           
            for indice in indices:
                for i in range(indice, fim+1):
                    # ver se nao tem antes
                    frase = so_tokens.copy()
                    termo = frase[indice:i+2]
                    frase.insert(indice, '<e1>')
                    #print(i)
                    
                    frase.insert(i+2, '</e1>')
                    frase_string=' '.join(frase).strip()
                    #print(frase_string)
                    #if frase_string not in combinacaoEntidades:
                    # ver se frase não termina com pontuacao ('.',',',';','-',')','(',']','[','/','"',)
                    devePular = 0
                    if '. </e1>' in frase_string or ', </e1>' in frase_string  or '; </e1>' in frase_string or '- </e1>' in frase_string  or ': </e1>' in frase_string  or '= </e1>' in frase_string  or '/ </e1>' in frase_string  or '( </e1>' in frase_string  or ') </e1>' in frase_string  or '[ </e1>' in frase_string  or '] </e1>' in frase_string  or ': </e1>' in frase_string or 'and </e1>' in frase_string or 'or </e1>' in frase_string:
                        devePular=1
                    if '<e1> .' in frase_string or '<e1> ,' in frase_string  or '<e1> ;' in frase_string or '<e1> -' in frase_string  or '<e1> :' in frase_string  or '<e1> =' in frase_string  or '<e1> /' in frase_string  or '<e1> (' in frase_string  or '<e1> )' in frase_string  or '<e1> [' in frase_string  or '<e1> ]' in frase_string  or '<e1> :' in frase_string  or '<e1> and' in frase_string  or '<e1> or' in frase_string:
                        devePular=1
                    if re.search("<e1> [0-9]* </e1>", frase_string):
                        devePular=1
                    pos_tagger_termo = tipoPostaggerTokens(termo, dicPosTagger)
                    if pos_tagger_termo not in lista_postaggers_entidades:
                        pulando_termos_postagger.append([termo, pos_tagger_termo])
                        devePular=1
                        
                    tem_frase = 0
                    for frase in combinacaoEntidades:
                        if frase[0] == frase_string:
                            tem_frase=''
                            break
                    if tem_frase==0 and devePular==0:
                        #print('inserindo:', frase_string)
                        #print('indice:', indice)
                        combinacaoEntidades.append([frase_string, 'O'])
    # shuffle no combinacaoEntidades
    if len(combinacaoEntidades)>0:
        random.shuffle(combinacaoEntidades)
        combinacaoEntidadesAll.append([' '.join(so_tokens).strip(), combinacaoEntidades])
    else:
        num_frases_sem_entidade = num_frases_sem_entidade+1
        #print("key sem entidade", key)
        #print('len(combinacaoEntidades):', len(combinacaoEntidades))
        #print(so_tokens)
    combinacaoEntidades = list()
    
    if (num % 1000) ==0:
    #if (num %3) == 0:
        print('key:', key)
        #break
print('numDescontinuas:', numDescontinuas)
print('erro_corpus:', erro_corpus)
print('num_frases_sem_entidade:', num_frases_sem_entidade)
#print(combinacaoEntidadesAll)

key: 999
key: 1999
key: 2999
key: 3999
key: 4999
key: 5999
key: 6999
key: 7999
key: 8999
key: 9999
key: 10999
key: 11999
key: 12999
key: 13999
key: 14999
key: 15999
key: 16999
key: 17999
numDescontinuas: 1395
erro_corpus: 1760
num_frases_sem_entidade: 1841


In [833]:
print(len(pulando_termos_postagger))
#print(pulando_termos_postagger)

174085


In [834]:
pulando_termos_postagger[:10]

[[['CD28', 'requires'], '-NN-VBZ'],
 [['5', '-', 'lipoxygenase', '.'], '-CD-NN-NN-.'],
 [['-', 'lipoxygenase', '.'], '-NN-NN-.'],
 [['lipoxygenase', '.'], '-NN-.'],
 [['CD28', 'surface', 'receptor', 'provides'], '-NN-NN-NN-VBZ'],
 [['surface', 'receptor', 'provides'], '-NN-NN-VBZ'],
 [['receptor', 'provides'], '-NN-VBZ'],
 [['interleukin', '-', '2', '('], '-NN-NN-CD-('],
 [['-', '2', '('], '-NN-CD-('],
 [['2', '('], '-CD-(']]

In [835]:
combinacaoEntidadesAll[:1]

[['IL - 2 gene expression and NF - kappa B activation through CD28 requires reactive oxygen production by 5 - lipoxygenase .',
  [['IL - <e1> 2 gene </e1> expression and NF - kappa B activation through CD28 requires reactive oxygen production by 5 - lipoxygenase .',
    'O'],
   ['IL - 2 gene expression and NF - kappa <e1> B </e1> activation through CD28 requires reactive oxygen production by 5 - lipoxygenase .',
    'O'],
   ['IL - 2 <e1> gene </e1> expression and NF - kappa B activation through CD28 requires reactive oxygen production by 5 - lipoxygenase .',
    'O'],
   ['IL - 2 gene expression and NF - kappa B activation through <e1> CD28 </e1> requires reactive oxygen production by 5 - lipoxygenase .',
    'PROTEIN'],
   ['IL - 2 gene expression and <e1> NF </e1> - kappa B activation through CD28 requires reactive oxygen production by 5 - lipoxygenase .',
    'O'],
   ['IL - 2 gene expression and NF - <e1> kappa B </e1> activation through CD28 requires reactive oxygen production b

In [744]:
# já ir gravando arquivos treinamento, test e dev...
# pra fazer um teste sem descontinuas
numTreinamento = len(combinacaoEntidadesAll)*0.8
numTreinamento = int(numTreinamento)
numTeste = (len(combinacaoEntidadesAll) - numTreinamento)/2
numTeste = int(numTeste)
numTotalEntidades=0
numTotalEntidadesTrain=0
numTotalEntidadesDev=0
numTotalEntidadesTest=0

print('numTreinamento:', numTreinamento)
print('numTeste:', numTeste)

f_train = open('genia.train', 'w')
f_dev = open('genia.dev', 'w')
f_test = open('genia.test', 'w')

for i, combinacaoEntidades in enumerate(combinacaoEntidadesAll):
    #print(dicSentences[i])
    frase = combinacaoEntidades[0]
    frases_entidade = combinacaoEntidades[1]
    if i<=numTreinamento:
        f_train.write(frase+'\n')
        for frase_entidade in frases_entidade:
            f_train.write(frase_entidade[1]+'\t'+frase_entidade[0]+'\n')
            numTotalEntidades=numTotalEntidades+1
            numTotalEntidadesTrain=numTotalEntidadesTrain+1
    elif i <= numTreinamento + numTeste:
        f_dev.write(frase+'\n')
        for frase_entidade in frases_entidade:
            f_dev.write(frase_entidade[1]+'\t'+frase_entidade[0]+'\n')
            numTotalEntidades=numTotalEntidades+1
            numTotalEntidadesDev=numTotalEntidadesDev+1
    else:
        f_test.write(frase+'\n')
        for frase_entidade in frases_entidade:
            f_test.write(frase_entidade[1]+'\t'+frase_entidade[0]+'\n')
            numTotalEntidades=numTotalEntidades+1
            numTotalEntidadesTest=numTotalEntidadesTest+1

f_train.close()
f_test.close()
f_dev.close()

print('numTotalEntidades:', numTotalEntidades)
print('numTotalEntidadesTrain:', numTotalEntidadesTrain)
print('numTotalEntidadesDev:', numTotalEntidadesDev)
print('numTotalEntidadesTest:', numTotalEntidadesTest)


numTreinamento: 13291
numTeste: 1661


### Agora, tratar descontinuas

In [836]:
combinacaoEntidades_desc = list()
combinacaoEntidadesAll_desc = list()
palavrasDescontinua=['AND NOT', 'AND/OR', 'AS WELL AS', 'AND', 'OR', 'BUT NOT', 'NEITHER NOR', 'THAN']
pulando_termos_postagger=[]
# agora, tratar descontinuas
for key in frasesComDescontinuas:
    entidades = dicSentencesComLabel[key][1].copy()
    for entidade in entidades:
        indices = entidade[1]
        if not areConsecutive(indices): # é descontinua
            tipo_entidade = entidade[2]
            texto_entidade = ' '.join(entidade[0])
            #print(texto_entidade)
            inicio=indices[0]
            fim=indices[-1]
            tokens = dicSentencesComLabel[key][0].copy()
            so_tokens = [t[0] for t in tokens]
            #print('so_tokens:', so_tokens)
            inicio_frase = so_tokens[:inicio]
            fim_frase = so_tokens[fim+1:-1] 
            #print('inicio_frase:', inicio_frase)
            #print('fim_frase:', fim_frase)
            meio_frase = '<e1> '+texto_entidade + ' </e1> '
            frase_final = ' '.join(inicio_frase)+' '+meio_frase+' '.join(fim_frase)+' .'
            frase_final_texto = ' '.join(inicio_frase)+' '+texto_entidade+' '.join(fim_frase)+' .'
            frase_final = replaceWhiteSpaces(frase_final)
            frase_final_texto = replaceWhiteSpaces(frase_final_texto)
            #print('frase_final_texto:', frase_final_texto)
            #print('frase_final:', frase_final)
            combinacaoEntidades_desc.append([frase_final, tipo_entidade])
            if len(combinacaoEntidades_desc)>0:
                combinacaoEntidadesAll_desc.append([frase_final_texto, combinacaoEntidades_desc])
            combinacaoEntidades_desc = list()
            '''
            #agora, as combinações possiveis
            # primeiro, gerar o termo todo
            menorIndice = entidade[1][0]
            maiorIndice = entidade[1][-1]
            novoIndice=[n for n in range(menorIndice, maiorIndice+1)]
            print(menorIndice)
            print(maiorIndice)
            print('novoIndice:', novoIndice)
            novoTexto=[]
            for token in tokens:
                for indice in novoIndice:
                    if token[1] == indice:
                        novoTexto.append(token[0]) # novo termo com o CC
                        
            novoTextoCC = ' '.join(novoTexto).lower()
            novoTextoCC = novoTextoCC.replace('and not', "and_not").replace('as well as', "as_well_as").replace('neither nor', "neither_no")
            #indices_cc = [palavra.lower() in novoTexto.split() for palavra in palavrasDescontinua]
            #indices2 = [txt.lower().split().index(palavra.lower()) for palavra in palavrasDescontinua]

            # segundo, ver onde está o CC (AND, OR, etc)
            indice_cc=0
            len_cc=0
            for cc in palavrasDescontinua:
                try:
                    indice_cc = novoTextoCC.split().index(cc.lower()) # pega o primero indice da primeira palavra CC
                    len_cc = len(cc.split())
                    break
                except:
                    continue
            
            # terceiro, fazer a combinação com as descontinuas:
            # tudo antes do CC + <pula um token> + tokens (a partir daqui, combinar)
            print('novoTexto[0:-2]:', novoTexto[0:len(novoTexto)])
            termos_antes_cc = novoTexto[:indice_cc]
            #termos_apos_cc = novoTexto[len_cc+3:-1] # nao pega o primeiro apos
            termos_apos_cc = novoTexto[len_cc+3:len(novoTexto)] # nao pega o primeiro apos
            #termos_ultimo_cc = novoTexto[-1] # nao pega o primeiro apos
            print('termos_antes_cc:', termos_antes_cc)
            print('termos_apos_cc:', termos_apos_cc)
            #print('termos_ultimo_cc:', termos_ultimo_cc)
            
            # concatena o termo antes, com os termos apos, mais a ultima palavra          
            # a primeira e ultima palavra tbm é um termo
            #nova_entidade = termos_antes_cc
            nova_entidade = []
            #for i, termo_apos in enumerate(termos_apos_cc):
            for i in range(len(termos_apos_cc), 0, -1):
                #nova_entidade = termos_antes_cc
                #for j in range(i, len(termos_apos_cc)):
                #nova_entidade = nova_entidade +[termos_apos_cc[j]]
                nova_entidade = [termos_apos_cc[i-1]]+nova_entidade
                #for j in range(i, 0, -1):
                    #print('j:',j)
                #    print('add ',termos_apos_cc[j-1])
                #    nova_entidade = [termos_apos_cc[j-1]]+nova_entidade
                #nova_entidade = nova_entidade
                #print('nova_entidade:', termos_antes_cc+nova_entidade)
                #nova_entidade=[]
                frase_final = ' '.join(inicio_frase)+' '+' '.join(termos_antes_cc+nova_entidade)+' '.join(fim_frase)
                frase_final_texto = ' '.join(inicio_frase)+' <e1> '+' '.join(termos_antes_cc+nova_entidade)+' </e1>'+' '.join(fim_frase)
                print('frase_final:', frase_final)
                print('frase_final_texto:', frase_final_texto)
                # só salva se nao tem ainda
                # e ver se está no padrão para salvar
                devePular = 0
                if '. </e1>' in frase_final_texto or ', </e1>' in frase_final_texto  or '; </e1>' in frase_final_texto or '- </e1>' in frase_final_texto  or ': </e1>' in frase_final_texto  or '= </e1>' in frase_final_texto  or '/ </e1>' in frase_final_texto  or '( </e1>' in frase_final_texto  or ') </e1>' in frase_final_texto  or '[ </e1>' in frase_final_texto  or '] </e1>' in frase_final_texto  or ': </e1>' in frase_final_texto or 'and </e1>' in frase_final_texto or 'or </e1>' in frase_final_texto:
                    devePular=1
                print('devePular:', devePular)
                if '<e1> .' in frase_final_texto or '<e1> ,' in frase_final_texto  or '<e1> ;' in frase_final_texto or '<e1> -' in frase_final_texto  or '<e1> :' in frase_final_texto  or '<e1> =' in frase_final_texto  or '<e1> /' in frase_final_texto  or '<e1> (' in frase_final_texto  or '<e1> )' in frase_final_texto  or '<e1> [' in frase_final_texto  or '<e1> ]' in frase_final_texto  or '<e1> :' in frase_final_texto  or '<e1> and' in frase_final_texto  or '<e1> or' in frase_final_texto:
                    devePular=1
                print('devePular:', devePular)
                if re.search("<e1> [0-9]* </e1>", frase_final_texto):
                    devePular=1
                print('devePular:', devePular)
                pos_tagger_termo = tipoPostaggerTokens(termo, dicPosTagger)
                if pos_tagger_termo not in lista_postaggers_entidades:
                    pulando_termos_postagger.append([termo, pos_tagger_termo])
                    devePular=1

                tem_frase = 0
                for frase in combinacaoEntidades:
                    if frase[0] == frase_string:
                        tem_frase=''
                        break
                print('tem_frase:', tem_frase)
                print('devePular:', devePular)
                if tem_frase==0 and devePular==0:
                    combinacaoEntidades_desc.append([frase_final, 'O'])
                    combinacaoEntidadesAll_desc.append([frase_final_texto, combinacaoEntidades_desc])
                combinacaoEntidades_desc= list()
            '''
            
    #break
    
combinacaoEntidadesAll_desc
#pulando_termos_postagger

[['This delay is associated with down - regulation of many erythroid cell - specific genes , including alpha - globin, band 3 , band 4.1 , and the erythroid cell - specific histone H5 .',
  [['This delay is associated with down - regulation of many erythroid cell - specific genes , including <e1> alpha - globin </e1> , band 3 , band 4.1 , and the erythroid cell - specific histone H5 .',
    'DNA']]],
 ['Transient transfection experiments show that several elements in the promoter - proximal region of the IL - 2R alpha gene contribute to IL - 1 responsiveness , most importantly an NF - kappa B site conserved in the human gene .',
  [['Transient transfection experiments show that several elements in the promoter - proximal region of the IL - 2R alpha gene contribute to IL - 1 responsiveness , most importantly an NF - kappa B site conserved in the <e1> human gene </e1> .',
    'DNA']]],
 ['Finally , the status of our current knowledge concerning the roles of transcription factors in the c

In [837]:
len(combinacaoEntidadesAll_desc)

1568

In [838]:
len(combinacaoEntidadesAll)

16614

In [840]:
# já ir gravando arquivos treinamento, test e dev...
# pra fazer um teste sem descontinuas
numTreinamento = len(combinacaoEntidadesAll)*0.8
numTreinamento = int(numTreinamento)
numTeste = (len(combinacaoEntidadesAll) - numTreinamento)/2
numTeste = int(numTeste)
numTotalEntidades=0
numTotalEntidadesTrain=0
numTotalEntidadesDev=0
numTotalEntidadesTest=0

print('numTreinamento:', numTreinamento)
print('numTeste:', numTeste)

f_train = open('genia.train', 'w')
f_dev = open('genia.dev', 'w')
f_test = open('genia.test', 'w')
f_dev_entidades = open('genia_entidades.dev', 'w')
f_test_entidades = open('genia_entidades.test', 'w')

for i, combinacaoEntidades in enumerate(combinacaoEntidadesAll):
    #print(dicSentences[i])
    frase = combinacaoEntidades[0]
    frases_entidade = combinacaoEntidades[1]
    if i<=numTreinamento:
        f_train.write(frase+'\n')
        for frase_entidade in frases_entidade:
            f_train.write(frase_entidade[1]+'\t'+frase_entidade[0]+'\n')
            numTotalEntidades=numTotalEntidades+1
            numTotalEntidadesTrain=numTotalEntidadesTrain+1
    elif i <= numTreinamento + numTeste:
        f_dev.write(frase+'\n')
        for frase_entidade in frases_entidade:
            f_dev.write(frase_entidade[1]+'\t'+frase_entidade[0]+'\n')
            f_dev_entidades.write(frase_entidade[1]+'\t'+frase_entidade[0]+'\n')
            numTotalEntidades=numTotalEntidades+1
            numTotalEntidadesDev=numTotalEntidadesDev+1
    else:
        f_test.write(frase+'\n')
        for frase_entidade in frases_entidade:
            f_test.write(frase_entidade[1]+'\t'+frase_entidade[0]+'\n')
            f_test_entidades.write(frase_entidade[1]+'\t'+frase_entidade[0]+'\n')
            numTotalEntidades=numTotalEntidades+1
            numTotalEntidadesTest=numTotalEntidadesTest+1

f_train.close()
f_test.close()
f_dev.close()
f_test_entidades.close()
f_dev_entidades.close()

print('numTotalEntidades:', numTotalEntidades)
print('numTotalEntidadesTrain:', numTotalEntidadesTrain)
print('numTotalEntidadesDev:', numTotalEntidadesDev)
print('numTotalEntidadesTest:', numTotalEntidadesTest)


numTreinamento: 13291
numTeste: 1661
numTotalEntidades: 165571
numTotalEntidadesTrain: 131376
numTotalEntidadesDev: 16986
numTotalEntidadesTest: 17209


In [ ]:
'''
combinacaoEntidades_desc = list()
combinacaoEntidadesAll_desc = list()
palavrasDescontinua=['AND NOT', 'AND/OR', 'AS WELL AS', 'AND', 'OR', 'BUT NOT', 'NEITHER NOR', 'THAN']

# agora, tratar descontinuas
for key in frasesComDescontinuas:
    entidades = dicSentencesComLabel[key][1].copy()
    for entidade in entidades:
        indices = entidade[1]
        if not areConsecutive(indices): # é descontinua
            tipo_entidade = entidade[2]
            inicio=indices[0]
            fim=indices[-1]
            tokens = dicSentencesComLabel[key][0].copy()
            so_tokens = [t[0] for t in tokens]
            inicio_frase = so_tokens[:inicio]
            fim_frase = so_tokens[fim+2:-1] # TODO - mais 2 ou 3?
            print('inicio_frase:', inicio_frase)
            print('fim_frase:', fim_frase)
            texto_entidade = ' '.join(entidade[0])
            meio_frase = '<e1> '+texto_entidade + ' </e1>'
            frase_final = inicio_frase+meio_frase+fim_frase
            frase_final_texto = inicio_frase+texto_entidade+fim_frase
            print('frase_final_texto:', frase_final_texto)
            print('frase_final:', frase_final)
            combinacaoEntidades_desc.append([frase_final, tipo_entidade])
            combinacaoEntidadesAll_desc.append([frase_final_texto, combinacaoEntidades_desc])
            #agora, as combinações possiveis
            # primeiro, gerar o termo todo
            menorIndice = entidade[1][0]
            maiorIndice = entidade[1][-1]
            novoIndice=[n for n in range(menorIndice, maiorIndice+1)]
            #print(menorIndice)
            #print(maiorIndice)
            #print('novoIndice:', novoIndice)
            novoTexto=[]
            for token in tokens:
                for indice in novoIndice:
                    if token[1] == indice:
                        novoTexto.append(token[0]) # novo termo com o CC
                        
            novoTextoCC = ' '.join(novoTexto).lower()
            novoTextoCC = novoTexto.replace('and not', "and_not").replace('as well as', "as_well_as").replace('neither nor', "neither_no")
            #indices_cc = [palavra.lower() in novoTexto.split() for palavra in palavrasDescontinua]
            #indices2 = [txt.lower().split().index(palavra.lower()) for palavra in palavrasDescontinua]

            # segundo, ver onde está o CC (AND, OR, etc)
            indice_cc=0
            len_cc=0
            for cc in palavrasDescontinua:
                try:
                    indice_cc = novoTextoCC.split().index(cc.lower()) # pega o primero indice da primeira palavra CC
                    len_cc = len(cc.split())
                    break
                except:
                    continue
            
            # terceiro, fazer a combinação com as descontinuas:
            # tudo antes do CC + <pula um token> + tokens (a partir daqui, combinar)
            termos_antes_cc = novoTexto[:indice_cc]
            termos_apos_cc = novoTexto[len_cc+1:-1]# nao pega o primeiro apos
            print('termos_antes_cc:', termos_antes_cc)
            print('termos_apos_cc:', termos_apos_cc)
            for i, termo_apos in enumerate(termos_apos_cc):
                nova_entidade = termos_antes_cc
                for j in range(i, len(termos_apos_cc)):                
                    nova_entidade += nova_entidade +' ' + termos_apos_cc[j]
                    print('nova_entidade:', nova_entidade)
            # só salva se nao tem ainda
            
            # quarto: gerar nova frase com a nova combinação (replace das tags especiaid)

    
'''

In [15]:
'''
#dicSentencesComLabel_tratandoDesc = {}

# soma as descontinuas numa entidade só
# ex: alfa and beta globina -> ENT, ENT, ENT, ENT (o 'AND' vira ENT)
# de 17 a 22
for fraseDescontinua in frasesComDescontinuas:
    entidades = dicSentencesComLabel[fraseDescontinua][1]
    tokens = dicSentencesComLabel[fraseDescontinua][0]
    #print(entidades)
    novasEnt = []
    for entidade in entidades:
        #print('entidade:', entidade)
        menorIndice = entidade[1][0]
        maiorIndice = entidade[1][-1]
        novoIndice=[n for n in range(menorIndice, maiorIndice+1)]
        #print(menorIndice)
        #print(maiorIndice)
        #print('novoIndice:', novoIndice)
        novoTexto=[]
        for token in tokens:
            for indice in novoIndice:
                if token[1] == indice:
                    novoTexto.append(token[0])
        #novoTexto = [token[0] for token in tokens where token[1] in novoIndice]
        #print('novoTexto:', novoTexto)
        novaEnt = [novoTexto, novoIndice]
        novasEnt.append(novaEnt)
    dicSentencesComLabel[fraseDescontinua][1] = novasEnt
'''